In [4]:
# import packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from itertools import chain

In [2]:
df = pd.read_csv('../res/input0130.csv')

In [3]:
scaler = StandardScaler()
starter = range(240,len(df)-1000,250)
all_results = []
all_evas = []
y_pred_clf = []
all_x_train = []
all_x_test = []
all_y_train = []
all_y_test = []
for train_begin in starter:
    train_end = train_begin + 750
    test_end = train_end + 250
    train_set = df[{'log_r','label'}][train_begin:train_end].reset_index()
    x_train, y_train = [], []
    x_train_set = list(chain.from_iterable(scaler.fit_transform(train_set['log_r'].values.reshape(-1,1))))
    for i in range(240,len(x_train_set)):
        x_train.append(x_train_set[i-240:i])
        y_train.append(train_set['label'][i])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1)) 
    
    # model = lstm_model()
    # model.fit(x_train,y_train,epochs=100,callbacks=[EarlyStopping(monitor='loss',patience=10)])

    x_test, y_test = [], []
    test_set = df[{'log_r','label'}][train_end-240:test_end].reset_index()
    x_test_set = list(chain.from_iterable(scaler.transform(test_set['log_r'].values.reshape(-1,1))))
    for i in range(240,len(x_test_set)):
        x_test.append(x_test_set[i-240:i])
        y_test.append(test_set['label'][i])
    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1)) 
    all_x_train.append(x_train)
    all_y_train.append(y_train)
    all_x_test.append(x_test)
    all_y_test.append(y_test)

    # all_results.append(model.predict(x_test))
    # all_evas.append(model.evaluate(x_test, y_test))
    # y_pred_clf.append(model.predict_classes(x_test))
    

In [4]:
from tqdm import tqdm
from keras.models import load_model, Model
from sa import *
from utils import *


Using TensorFlow backend.


In [5]:
model = load_model('lstm_r.h5')
default_upper_bound = 2000
default_n_bucket = 1000
default_n_classes = 2
class Args(): #创建一个类
    def __init__(self): #定义初始化信息。
        self.is_classification = True
        self.save_path = './tmp/'
        self.d = 'lstm_r'
        self.num_classes = 2
        self.lsa = True
        self.dsa = True
        self.target = 'none'
        self.batch_size = 128
        self.var_threshold = 1e-5
        self.upper_bound = 2000
        self.n_bucket = 1000
        self.num_classes = 10
        self.is_classification = True
args = Args()
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 240, 25)           2700      
_________________________________________________________________
lstm_4 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 52        
Total params: 7,852
Trainable params: 7,852
Non-trainable params: 0
_________________________________________________________________


In [6]:
layer_names = ['lstm_3']

In [7]:
test_lsa = fetch_lsa(model, x_train, x_test, "test", layer_names, args)
target_lsa = fetch_lsa(model, x_train, x_test, "target", layer_names, args)
# target_lsa = fetch_lsa(model, x_train, x_target, args.target, layer_names, args)
target_cov = get_sc(
    np.amin(target_lsa), default_upper_bound, default_n_bucket, target_lsa
)

auc = compute_roc_auc(test_lsa, target_lsa)
print(infog("ROC-AUC: " + str(auc * 100)))

print(infog("LSA coverage: " + str(target_cov)))

[train] Model serving
510/510 [==============================] - 1s 1ms/step
[train] Processing ATs
Layer: lstm_3
train ATs is saved at ./tmp/lstm_r_train_lstm_3_ats.npy
[test] Model serving
250/250 [==============================] - 0s 1ms/step
[test] Processing ATs
Layer: lstm_3
test ATs is saved at ./tmp/lstm_r_test_lstm_3_ats.npy


KeyError: 2

In [25]:
train_ats, train_pred, target_ats, target_pred = _get_train_target_ats(
    model, x_train, x_test, "test", layer_names, args
)

[train] Model serving
510/510 [==============================] - 1s 1ms/step
[train] Processing ATs
Layer: lstm_33
train ATs is saved at ./tmp/lstm_r_train_lstm_33_ats.npy
[test] Model serving
250/250 [==============================] - 0s 2ms/step
[test] Processing ATs
Layer: lstm_33
test ATs is saved at ./tmp/lstm_r_test_lstm_33_ats.npy


In [24]:
train_ats, train_pred = get_ats(
    model,
    x_train,
    "train",
    layer_names,
    num_classes=args.num_classes,
    is_classification=args.is_classification,
    save_path='./tmp/savepath',
)

[train] Model serving
510/510 [==============================] - 1s 1ms/step
[train] Processing ATs
Layer: lstm_33


In [28]:
def fetch_lsa(model, x_train, x_target, target_name, layer_names, args):
    """Likelihood-based SA

    Args:
        model (keras model): Subject model.
        x_train (list): Set of training inputs.
        x_target (list): Set of target (test or[] adversarial) inputs.
        target_name (str): Name of target set.
        layer_names (list): List of selected layer names.
        args: Keyboard args.

    Returns:
        lsa (list): List of lsa for each target input.
    """

    prefix = info("[" + target_name + "] ")
    train_ats, train_pred, target_ats, target_pred = _get_train_target_ats(
        model, x_train, x_target, target_name, layer_names, args
    )

    class_matrix = {}
    if args.is_classification:
        for i, label in enumerate(train_pred):
            label = label[0]
            if label not in class_matrix:
                class_matrix[label] = []
            class_matrix[label].append(i)

    kdes, removed_cols = _get_kdes(train_ats, train_pred, class_matrix, args)

    lsa = []
    print(prefix + "Fetching LSA")
    if args.is_classification:
        for i, at in enumerate(tqdm(target_ats)):
            label = target_pred[i]
            kde = kdes[label]
            lsa.append(_get_lsa(kde, at, removed_cols))
    else:
        kde = kdes[0]
        for at in tqdm(target_ats):
            lsa.append(_get_lsa(kde, at, removed_cols))

    return lsa